In [ ]:
###Create greyscale images###
import os
from os import path, rename
from PIL import Image
import PIL.ImageOps

Output = '/media/john/My Passport1/Pyradiomics_Input_Output/Output/FullEcho/Grey_Full_Output/image/'

for filename in os.listdir('/media/john/My Passport1/Pyradiomics_Input_Output/Output/FullEcho/IVS/'):
    if filename.endswith('original_IVS.png'):
        ##Open Mask File##
        mask = Image.open(os.path.join('/media/john/My Passport1/Pyradiomics_Input_Output/Output/FullEcho/IVS', filename)).convert('L')
        mask.save(path.join(Output, filename))

In [ ]:
###Sort images into folders based on CSV###
import pandas as pd
import shutil
import os
from shutil import move
import csv
import os

with open('/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/CHOICE_groundtruth.csv', newline='') as csvfile:
    linereader = csv.reader(csvfile, delimiter=',')
    for row in linereader:
        name = row[0]
        label = row[1]
        if label == 'o':
            old_img_path = os.path.join("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/image/", name)
            new_img_path = os.path.join("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full/class1/", name)
            move(old_img_path, new_img_path)
        if label == 'l':
            old_img_path = os.path.join("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/image/", name)
            new_img_path = os.path.join("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full/class2/", name)
            move(old_img_path, new_img_path)

In [ ]:
###Random Splitting of folders into training/testing/validation###
import splitfolders
splitfolders.ratio('/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full/', output="/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/output/", seed=100, ratio=(0.6, 0.2, 0.2)) 

In [ ]:
#Create masked image###
import numpy as np
import cv2

import os
from os import path, rename
from PIL import Image
import PIL.ImageOps

Output = '/media/john/My Passport1/Pyradiomics_Input_Output/Output/Camp_India/Grey_PW/input/'

for filename in os.listdir('/media/john/My Passport1/Pyradiomics_Input_Output/Output/Camp_India/Grey_Full/original_image/'):
    if filename.endswith('original_IVS.png'):
        ##Open Image File##
        src1 = cv2.imread(os.path.join('/media/john/My Passport1/Pyradiomics_Input_Output/Output/Camp_India/Grey_Full/original_image/', filename)) 
    if filename.endswith('segmentation_PW.png'):
        ##Open Mask File##
        src1_mask = cv2.imread(os.path.join('/media/john/My Passport1/Pyradiomics_Input_Output/Output/Camp_India/Grey_Full/original_image/', filename))
        mask_out=cv2.subtract(src1_mask,src1)
        mask_out=cv2.subtract(src1_mask,mask_out)
        differenceImage = Image.fromarray(mask_out)
        differenceImage.save(path.join(Output, filename))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.preprocessing.image import image_dataset_from_directory
from keras import  layers
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
%autosave 5
print(tf.__version__)

In [ ]:
image_size = (331, 331)
batch_size = 32
input_shape=image_size + (3,)
# number of output classes
num_classes = 1

class_names  =['class1','class2']

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/quincy/Documents/Research/HVI/Radiomics - Automated/FINAL/DeepLearning/Grey_Full_Output_ASE-REWARD/output/train', 
    validation_split=0.2, subset='training',
     seed=100,
    image_size=image_size, batch_size=batch_size,
    labels= "inferred",
    class_names= class_names)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/quincy/Documents/Research/HVI/Radiomics - Automated/FINAL/DeepLearning/Grey_Full_Output_ASE-REWARD/output/val', validation_split=0.2, subset='validation', seed=100,
    image_size=image_size, 
     labels= "inferred",
     class_names= class_names,
    batch_size=batch_size)

print(input_shape)

In [ ]:
#print(next(iter(val_ds.take(1))))

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/quincy/Documents/Research/HVI/Radiomics - Automated/FINAL/DeepLearning/Grey_Full_Output_ASE-REWARD/output/test', seed=100,
    image_size=image_size, 
     labels= "inferred",
     class_names= class_names,
     batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
#for images, labels in train_ds.take(1):
for i, (images, labels) in enumerate(train_ds.take(9)):
#  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    #plt.title(class_names[labels[i]])
    plt.title("label  {n} : {s}".format(n=labels[i].numpy(), s=class_names[labels[i]]),color='b')  
    #plt.title("Label %s : %s" % (labels[i].numpy(), class_names[labels[i]]))
    plt.axis("off")
#    plt.savefig("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/Labeled_Images.png")

In [ ]:
import itertools
import glob
train_filenames = glob.glob('/Users/quincy/Documents/Research/HVI/Radiomics - Automated/FINAL/DeepLearning/Grey_Full_Output_ASE-REWARD/output/train/*/*')
test_filenames = glob.glob('/Users/quincy/Documents/Research/HVI/Radiomics - Automated/FINAL/DeepLearning/Grey_Full_Output_ASE-REWARD/output/test/*/*')
#filenames[0:5]
rand_image = [train_filenames[i] for i in np.random.randint(len(train_filenames), size=10)]
#rand_image = list(itertools.compress(filenames, np.random.randint(len(filenames), size=10)))
#filenames[[0,1]]
print(rand_image[0:2])
print(len(train_filenames))
print(len(test_filenames))

In [ ]:
train_labels = []
for filename in train_filenames:
   if "class1" in filename:
      train_labels.append("class1")
   elif "class2" in filename:
     train_labels.append("class2")

from collections import Counter
print(Counter(train_labels).keys() )# equals to list(set(words))
print(Counter(train_labels).values()) # counts the elements' frequency    
print(Counter(train_labels)['class1'])
print(Counter(train_labels)['class2'])

In [ ]:
test_labels = []
for filename in test_filenames:
   if "class1" in filename:
      test_labels.append("class1")
   elif "class2" in filename:
     test_labels.append("class2")

from collections import Counter
print(Counter(test_labels).keys() )# equals to list(set(words))
print(Counter(test_labels).values()) # counts the elements' frequency    
print(Counter(test_labels)['class1'])
print(Counter(test_labels)['class2'])

In [ ]:
label = []
for filename in rand_image:
   if "class1" in filename:
      label.append("class1")
   elif "class2" in filename:
     label.append("class2")

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
fig, axs = plt.subplots(2,5, figsize=(15, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)

axs = axs.ravel()

for i in range(10):

    axs[i].imshow(mpimg.imread(rand_image[i]))
    axs[i].set_title(label[i])
#plt.savefig("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/Labeled_Images2.png")    

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [ ]:
data_augmentation =tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomContrast(0.01)
])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
#for images, _ in train_ds.take(1):
for i, (images, _ ) in enumerate(train_ds.take(9)):
  #for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype('uint8'))
    plt.title("label  {n} : {s}".format(n=_[0].numpy(), s=class_names[_[0]]),color='b') 
    plt.axis('off')
#plt.savefig("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/Labeled_Images3.png")

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def create_model():
    base_model = tf.keras.applications.NASNetLarge(
    weights="imagenet",
    input_shape=input_shape,
    include_top=False,
     )

# Freeze the base_model
    base_model.trainable = False
    optimizer=keras.optimizers.Adam(1e-3)

# Create new model on top
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    norm_layer = tf.keras.layers.LayerNormalization(axis=1)
    x = norm_layer(x)
    x = base_model(x, training=False)
    x = keras.layers.Dense(252, activation='relu')(x)
    x = keras.layers.Dense(126, activation='sigmoid')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)
    
# A Dense classifier with a single unit (binary classification)
    outputs = tf.keras.layers.Dense(units=num_classes, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=optimizer,
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=METRICS)  #[keras.metrics.BinaryAccuracy()]
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

In [ ]:
base_model = tf.keras.applications.NASNetLarge(
    weights="imagenet",
    input_shape=input_shape,
    include_top=False,
     )
base_model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)
# to_file='/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/Workflow.png')

In [ ]:
import visualkeras
from PIL import ImageFont

visualkeras.layered_view(model, legend=True).show() # display using your system viewer
#visualkeras.layered_view(model, to_file='output.png') # write to disk
#visualkeras.layered_view(model, legend=True, to_file='/Users/quincy/Documents/Research/HVI/Radiomics - Automated/FINAL/Submission/Nature Cardiovascular Research/Figures/NASNetLarge_output.png') # write and show

#visualkeras.layered_view(model)

In [ ]:
print(Counter(train_labels)['class1'])
print(Counter(train_labels)['class2'])


initial_bias = np.log([Counter(train_labels)['class2']/Counter(train_labels)['class1']])
initial_bias

In [ ]:
# Scaling by total/2 can keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / Counter(train_labels)['class1'])*(len(train_labels))/2.0 
weight_for_1 = (1 / Counter(train_labels)['class2'])*(len(train_labels))/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
import os
# Include the epoch in the file name (uses `str.format`)
#checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)



checkpoint_p= "/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/"
checkpoint_path = os.path.join(checkpoint_p, "cp.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 monitor='val_auc',
                                                 save_best_only=True,
                                                 save_freq=10,
                                                verbose=0 )

# Keep only a single checkpoint, the best over test accuracy.
#cp_callback  = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                            monitor='val_accuracy',
#                            verbose=0,
#                            save_best_only=True,
#                            mode='max')


epochs = 20

#verbose =2 print a lot of information
early_stopping_monitor = tf.keras.callbacks.EarlyStopping(patience = 20 ,
                                                          monitor = "val_auc", 
                                                        mode="max",
                                                          verbose = 0,
                                                          restore_best_weights=True)
# Train the model with the new callback
history = model.fit(train_ds, 
          #train_labels,  
          epochs=epochs,
          class_weight=class_weight,
           #steps_per_epoch= len(train_filenames)// batch_size, #run out of data
          #validation_data=(test_images,test_labels),
          validation_data=val_ds,
          verbose=1,
          callbacks=[cp_callback,early_stopping_monitor])  # Pass callback to training

In [ ]:
#def plot_loss(history, label, n):
  # Use a log scale to show the wide range of values.
#  plt.plot(history.epoch,  history.history['loss'],
#               color=colors[n], label='Train_loss '+label)
#  plt.plot(history.epoch,  history.history['val_loss'],
 #         color=colors[n+1], label='Val_loss '+label,
 #         linestyle="--")
 # plt.xlabel('Epoch')
 # plt.ylabel('Loss')
 # plt.legend()
  #plt.savefig('acc.png')
model.save('/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/normlayer_model.h5')

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss','auc']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])


plt.savefig("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/Epochs1.png")    

In [ ]:
def plot_metrics(history):
  mpl.rcParams['figure.figsize'] = (12, 10)
  colors = plt.rcParams['axes.prop_cycle'].by_key()['color']  
  #metrics =  ['loss', 'auc', 'precision', 'recall']
  metrics =['precision', 'recall', 'accuracy', 'loss','auc']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(3,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[1], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    #if metric == 'loss':
    #  plt.ylim([0, plt.ylim()[1]])
    #elif metric == 'auc':
    #  plt.ylim([0.8,1])
    #else:
    #  plt.ylim([0,1])

    plt.legend()

In [ ]:
plot_metrics(history )
plt.savefig("/media/john/My Passport1/Pyradiomics_Input_Output/Output/CHOICE/Grey_Full_Output/Epochs2.png")  

In [ ]:
# Evaluate the restored model on the valdatio set
loss,tp,fp,tn,fn,accuracy,precision,recall,auc = model.evaluate(val_ds, verbose=2)
print(' model accuracy: {:5.2f}%'.format(100*accuracy))
print(' model Precision: {:5.2f}%'.format(100*precision))
print(' model Recall: {:5.2f}%'.format(100*recall))
print(' model True Positive: {:5.2f}'.format(tp))
print(' model True Negative: {:5.2f}'.format(tn))
print(' model False Positive: {:5.2f}'.format(fp))
print(' model False Negative: {:5.2f}'.format(fn))
print(' model loss: {:5.2f}'.format(loss))
print(' model AUC: {:5.2f}%'.format(100*auc))

In [ ]:
#loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
#loss, acc = model.evaluate(val_ds, verbose=2)
#print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

#print(new_model.predict(test_images).shape)

predictions = model.predict(test_ds, batch_size=batch_size)


test_labels_num =  [1 if x =='class2' else 0 for x in test_labels]

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
p=0.5

def PlotConfusionMatrix(y_test,pred,y_test_normal,y_test_pneumonia,label):

    cfn_matrix = confusion_matrix(y_test,pred)
    cfn_norm_matrix = np.array([[1.0 / y_test_normal,1.0/y_test_normal],[1.0/y_test_pneumonia,1.0/y_test_pneumonia]])
    norm_cfn_matrix = cfn_matrix * cfn_norm_matrix

    #colsum=cfn_matrix.sum(axis=0)
    #norm_cfn_matrix = cfn_matrix / np.vstack((colsum, colsum)).T

    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(1,2,1)
    sns.heatmap(cfn_matrix,cmap='magma',linewidths=0.5,annot=True,ax=ax)
    #tick_marks = np.arange(len(y_test))
    #plt.xticks(tick_marks, np.unique(y_test), rotation=45)
    plt.title('Confusion Matrix',color='b')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
 #   plt.savefig('/content/drive/My Drive/Colab Notebooks/ComputerVision/cm_' +label +  '.png')
        
    ax = fig.add_subplot(1,2,2)
    sns.heatmap(norm_cfn_matrix,cmap=plt.cm.Blues,linewidths=0.5,annot=True,ax=ax)

    plt.title('Normalized Confusion Matrix',color='b')
    plt.ylabel('Real Classes')
    plt.xlabel('Predicted Classes')
#    plt.savefig('/content/drive/My Drive/Colab Notebooks/ComputerVision/cm_norm' +label +  '.png')
    plt.show()
    
    print('---Classification Report---')
    print(classification_report(y_test,pred))
    

y_test_normal,y_test_pneumonia = np.bincount(test_labels_num)
y_pred= np.where(predictions<p,0,1 )


PlotConfusionMatrix(test_labels_num,y_pred,y_test_normal,y_test_pneumonia,label='classification')


In [ ]:
#loss, acc, prec, rec = model.evaluate(test_ds)

# Evaluate the restored model on the test set
loss,tp,fp,tn,fn,accuracy,precision,recall,auc =  model.evaluate(test_ds, verbose=2)
print(' model accuracy: {:5.2f}%'.format(100*accuracy))
print(' model Precision: {:5.2f}%'.format(100*precision))
print(' model Recall: {:5.2f}%'.format(100*recall))
print(' model True Positive: {:5.2f}'.format(tp))
print(' model True Negative: {:5.2f}'.format(tn))
print(' model False Positive: {:5.2f}'.format(fp))
print(' model False Negative: {:5.2f}'.format(fn))
print(' model loss: {:5.2f}'.format(loss))
print(' model AUC: {:5.2f}%'.format(100*auc))